In [1]:
%pwd

'/home/ahmed/CV_ADD_PROJECT/AIEnginear/AI-Medical-Assistant/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/home/ahmed/CV_ADD_PROJECT/AIEnginear/AI-Medical-Assistant'

In [4]:
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader

loader = DirectoryLoader(
    path="Data",
    glob= "*.pdf" ,# "**/*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()
print(f"Loaded pages: {len(documents)}")


/home/ahmed/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:45<00:00, 45.96s/it]

Loaded pages: 637


In [5]:
# documents

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)
print(f"Chunks created: {len(chunks)}")


Chunks created: 3428


In [8]:
# chunks

In [13]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small"
# )

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-l6-v2")

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



In [11]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "medical-rag"

# Check if index exists
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # Match your embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )



In [14]:
from langchain_pinecone import PineconeVectorStore
# Load documents into LangChain Pinecone vectorstore
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)

In [15]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [16]:
retrieved_docs = retriever.invoke("What is Abdominal ultrasound?")
retrieved_docs

[Document(id='3ee680b0-3fc5-432c-b211-c5f16415f1d1', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 17.0, 'page_label': '18', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='the quality of the equipment, the operator and other fac-\ntors. More involved techniques such as multiple M-mode\nand Doppler-enhanced scans, or cases where the targets\nnot well defined in advance, generally take more time\nand are more expensive.\nRegardless of the type of scan used and the potential\ndifficulties encountered, ultrasound remains faster and\nless expensive than computed tomography scans (CT),\nits primary rival in abdominal imaging. Furthermore, as\nabdominal ultrasounds are generally undertaken as\n“medically necessary” procedures designed to detect the\npresence of suspected abnormalities, they are covered\nGALE ENCYCLOPEDIA OF MEDICINE 24\nAbdominal ultras

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model= "gpt-4.1-nano", ## "gpt-4o-mini",  ## "gpt-5-nano",  # OpenAI chat model
    temperature=0.4, 
    max_tokens=500
    )


In [35]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.prompts import PromptTemplate

prompt_template = """
You are a helpful Medical AI assistant. Answer questions based on the provided context.

CONTEXT:
{context}

QUESTION:
{input}

INSTRUCTIONS:
1. Use only information from the context
2. If context doesn't contain answer, say "I don't know"
3. Keep answer concise (4-6 sentences)
4. Use simple language

ANSWER:
"""

prompt_template_obj = PromptTemplate(
    input_variables=["context", "input"],
    template=prompt_template
)

# create the stuff chain
combine_chain = create_stuff_documents_chain(llm, prompt_template_obj)

# create the retrieval chain
chain = create_retrieval_chain(retriever, combine_chain)



In [36]:
response = chain.invoke({"input": "whats Acupuncture?"})
print(f"🤖 {response['answer']}...")

🤖 Acupuncture is a treatment that involves inserting needles into specific points on the body. It is used to help with many health problems, such as allergies, respiratory issues, and pain. It has been used for a long time in Chinese medicine. Although it is considered safe, it is important to have it done by a knowledgeable and trained professional. Some believe it works by affecting the nervous system and releasing pain-relieving chemicals called endorphins....


In [37]:
response = chain.invoke({"input": "whats Acoustic neuroma?"})
print(f"🤖 {response['answer']}...")

🤖 Acoustic neuroma is a benign tumor that develops from the myelin sheath surrounding the vestibular nerve. It is also called vestibular schwannoma. The tumor grows near the inner ear and can occur on one side (unilateral) or both sides (bilateral). As it grows, it may cause symptoms like dizziness, imbalance, facial pain, or numbness. Larger tumors can press on nearby cranial nerves, leading to additional problems....


In [32]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.prompts import PromptTemplate,ChatPromptTemplate

# Use ChatPromptTemplate for better compatibility with ChatOpenAI
system_prompt = (
    "You are a helpful Medical AI assistant. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say I don't know. "
    "Keep the answer concise (4-6 sentences).\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the chains using the standard recommended way
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is acupuncture?"})
print(f"🤖 {response['answer']}...")

🤖 Acupuncture is a traditional Chinese medicine practice that involves inserting fine needles into specific points on the body to treat a wide variety of illnesses and health conditions. It is used to address issues such as allergies, respiratory and gastrointestinal disorders, gynecological problems, nervous conditions, and childhood illnesses, among others. Although mainstream medicine has been slow to accept it due to the difficulty in scientifically understanding its mechanisms, some theories suggest it works by stimulating the nervous system and releasing endorphins. Acupuncture is generally considered safe when performed by a trained and knowledgeable practitioner....


In [33]:
response = rag_chain.invoke({"input": "whats Acoustic neuroma?"})
print(f"🤖 {response['answer']}...")

🤖 Acoustic neuroma, also known as vestibular schwannoma, is a benign tumor that develops from the myelin sheath surrounding the vestibular nerve. It most often occurs where the vestibulocochlear nerve passes through a bony canal connecting the brain and inner ear. These tumors can be unilateral or bilateral, with unilateral cases usually occurring spontaneously and bilateral cases often linked to hereditary conditions. As the tumor grows, it can press on nearby cranial nerves, causing symptoms like dizziness, vertigo, facial pain, numbness, or weakness....


In [34]:
response

{'input': 'whats Acoustic neuroma?',
 'context': [Document(id='300dec0b-6c90-40ad-9438-8c1a2efbe194', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 44.0, 'page_label': '45', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='Resources\nBOOKS\nFilipo, R., and Barbara Maurizio Acoustic neuroma: trends and\ncontroversies: proceedings of the Symposium Acoustic\nNeuroma: Trends and Controversies, Rome, Italy, Novem-\nber 13–15, 1997. The Hague, Netherlands: Kugler,1999.\nMalis, Leonard Acoustic Neuroma New York: Elsevier, 1998.\nRoland, Peter, and Bradley Marple. Diagnosis and Manage-\nment of Acoustic Neuroma (Sipac).Alexandria, V A:\nAmerican Academy of Otolaryngology—Head and Neck\nSurvey Foundation, 1998.\nPERIODICALS\nBroad, R. W. “Management of Acoustic Neuroma.” In New\nEngland Journal of Medicine. 340(14) (8 April\n1999):1119.\nLederman G, E. Arbi